In [37]:
using LazySets
using LazySets: isapprox, isapproxzero

function _two_points_2d!(points)

    # special case, see #876
    p1, p2 = points[1], points[2]
    if p1 == p2
        # check for redundancy
        pop!(points)
    elseif p1 <= p2
        nothing
    else
        points[1], points[2] = p2, p1
    end
    return points
end

function _three_points_2d!(points::AbstractVector{<:AbstractVector{N}}) where {N<:Real}
    # Algorithm: the function takes three points and uses the formula
    #            from here: https://stackoverflow.com/questions/2122305/convex-hull-of-4-points/2122620#2122620
    #            to decide if the points are ordered in a counter-clockwise fashion or not, the result is saved 
    #            in the 'turn' boolean, then returns the points in ordered ccw acting according to 'turn'. For the
    #            cases where the points are collinear we pass the points with the minimum and maximum first
    #            component to the function for two points in 2d(_two_points_2d), if those are equal, we do the same
    #            but with the second component.
    A, B, C = points[1], points[2], points[3]
    turn = (A[2] - B[2]) * C[1] + (B[1] - A[1]) * C[2] + (A[1] * B[2] - B[1] * A[2])

    if isapproxzero(turn)
        # ABC are collinear
        if isapprox(A[1], B[1]) && isapprox(B[1], C[1]) && isapprox(C[1], A[1])
            # points are approximately equal in their first component
            if isapprox(A[2], B[2]) && isapprox(B[2], C[2]) && isapprox(C[2], A[2])
                # all points are approximately equal
                pop!(points)
                pop!(points)
                return points
            else
                points[1], points[2] = points[argmin([A[2], B[2], C[2]])], points[argmax([A[2], B[2], C[2]])]
            end
        else
            points[1], points[2] = points[argmin([A[1], B[1], C[1]])], points[argmax([A[1], B[1], C[1]])]
        end
        pop!(points)
        _two_points_2d!(points)
    elseif turn < zero(N)
        # ABC is CW
        points[1], points[2], points[3] = C, B, A
    end
    # else ABC is CCW => nothing to do
    return points
end


function _four_points_2d!(points::AbstractVector{<:AbstractVector{N}}) where {N<:Real}
    A, B, C, D = points[1], points[2], points[3], points[4]
    tri_ABC = (A[2] - B[2]) * C[1] + (B[1] - A[1]) * C[2] + (A[1] * B[2] - B[1] * A[2])
    tri_ABD = (A[2] - B[2]) * D[1] + (B[1] - A[1]) * D[2] + (A[1] * B[2] - B[1] * A[2])
    tri_BCD = (B[2] - C[2]) * D[1] + (C[1] - B[1]) * D[2] + (B[1] * C[2] - C[1] * B[2])
    tri_CAD = (C[2] - A[2]) * D[1] + (B[1] - C[1]) * D[2] + (C[1] * A[2] - A[1] * C[2])
    
    function collinear_case(A, B, C, D)
        # A, B and C collinear, D is the extra point
        if isapprox(A[1], B[1]) && isapprox(B[1], C[1]) && isapprox(C[1], A[1])
            # points are approximately equal in their first component
            if isapprox(A[2], B[2]) && isapprox(B[2], C[2]) && isapprox(C[2], A[2])
                # the three points are approximately equal
                points[1], points[2] = A, D
                pop!(points)
                pop!(points)
                return _two_points_2d!(points)
            else
                # assign the points with max and min value in their second component to the
                # firsts points and the extra point to the third place, then pop the point that was in the middle
                points[1], points[2], points[3] = points[argmin([A[2], B[2], C[2]])], points[argmax([A[2], B[2], C[2]])], D
                pop!(points)
            end
        else
            # assign the points with max and min value in their first component to the
            # firsts points and the extra point to the third place, then pop the point that was in the middle
            points[1], points[2], points[3] = points[argmin([A[1], B[1], C[1]])], points[argmax([A[1], B[1], C[1]])], D
            pop!(points)
        end
        return _three_points_2d!(points)
    end
    
    if isapproxzero(tri_ABC)
        return collinear_case(A, B, C, D)
    end
    if isapproxzero(tri_ABD)
        return collinear_case(A, B, D, C)
    end
    if isapproxzero(tri_BCD)
        return collinear_case(B, C, D, A)
    end
    if isapproxzero(tri_CAD)
        return collinear_case(C, A, D, B)
    end
    
    # ABC  ABD  BCD  CAD  hull
    # ------------------------
    #  +    +    +    +   ABC
    #  +    +    +    -   ABCD
    #  +    +    -    +   ABDC
    #  +    +    -    -   ABD
    #  +    -    +    +   ADBC
    #  +    -    +    -   BCD
    #  +    -    -    +   CAD
    #  +    -    -    -   [should not happen]
    #  -    +    +    +   [should not happen]
    #  -    +    +    -   ACD
    #  -    +    -    +   DBC
    #  -    +    -    -   DBCA
    #  -    -    +    +   ADB
    #  -    -    +    -   ACDB
    #  -    -    -    +   ADCB
    #  -    -    -    -   ACB
    
    if tri_ABC > zero(N)
        if tri_ABD > zero(N)
            if tri_BCD > zero(N)
                if tri_CAD > zero(N)
                    points[1], points[2], points[3] = A, B, C # +    +    +    +   ABC
                    pop!(points)
                else
                    points[1], points[2], points[3], points[4] = A, B, C, D # +    +    +    -   ABCD
                end
            else
                if tri_CAD > zero(N)
                    points[1], points[2], points[3], points[4] = A, B, D, C #  +    +    -    +   ABDC
                else
                    points[1], points[2], points[3] = A, B, D #  +    +    -    -   ABD
                    pop!(points)
                end
            end
        else
            if tri_BCD > zero(N)
                if tri_CAD > zero(N)
                    points[1], points[2], points[3], points[4] = A, D, B, C #  +    -    +    +   ADBC
                else
                    points[1], points[2], points[3] = B, C, D #  +    -    +    -   BCD
                    pop!(points)
                end
            else
                if tri_CAD > zero(N)
                    points[1], points[2], points[3] = C, A, D #  +    -    -    +    CAD
                    pop!(points)
                else
                    #  +    -    -    -   [should not happen]
                end
            end
        end
    else
        if tri_ABD > zero(N)
            if tri_BCD > zero(N)
                if tri_CAD > zero(N)
                    #  -    +    +    +   [should not happen]
                else
                    points[1], points[2], points[3] = A, C, D #  -    +    +    -   ACD
                    pop!(points)
                end
            else
                if tri_CAD > zero(N)
                    points[1], points[2], points[3] = D, B, C #  -    +    -    +   DBC
                    pop!(points)
                else
                    points[1], points[2], points[3], points[4] = D, B, C, A #  -    +    -    -   DBCA
                end
            end
        else
            if tri_BCD > zero(N)
                if tri_CAD > zero(N)
                    points[1], points[2], points[3] = A, D, B #  -    -    +    +   ADB
                    pop!(points)
                else
                    points[1], points[2], points[3], points[4] = A, C, D, B #  -    -    +    -   ACDB
                end
            else
                if tri_CAD > zero(N)
                    points[1], points[2], points[3], points[4] = A, D, C, B#  -    -    -    +   ADCB
                else
                    points[1], points[2], points[3] = A, C, B #  -    -    -    -   ACB
                end
            end
        end
        return points
    end
end

_four_points_2d! (generic function with 1 method)

In [38]:
using BenchmarkTools#, Plots
points = [[-1.0, 1.0], [1.0, -1.0], [1.0, 1.0], [0.0, 0.0]]
#A, B, C, D = points[1], points[2], points[3], points[4]
#tri_ABD = (A[2] - B[2]) * D[1] + (B[1] - A[1]) * D[2] + (A[1] * B[2] - B[1] * A[2])
#isapproxzero(tri_ABD)
#plot(VPolygon(_four_points_2d!(points)))
@btime _four_points_2d!(copy($points))
#_four_points_2d!(points)

  208.483 ns (4 allocations: 352 bytes)


3-element Array{Array{Float64,1},1}:
 [-1.0, 1.0]
 [1.0, -1.0]
 [1.0, 1.0] 